# Core Zipcodes by Municipal Boundary (TIGER/Line + ZCTA)

This notebook produces core ZIP codes (ZCTAs) whose centroids fall strictly within each city's municipal boundary using US Census TIGER/Line Place polygons and ZCTA polygons.

- Input: `top_100_cities.csv`
- Output: `top_100_cities_core_zipcodes.csv` (ZIPCODES column added)
- Method: Spatial join of ZCTA centroids within Place polygons (incorporated/consolidated places)


In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union
from tqdm import tqdm

DATA_DIR = 'data_tiger'
os.makedirs(DATA_DIR, exist_ok=True)

print('Env ready')


In [ ]:
# Load cities list
cities = pd.read_csv('top_100_cities.csv')
# Normalize
cities['CITY_NORM'] = cities['CITY'].str.upper().str.replace('\.', '', regex=False).str.strip()
cities['STATE_NORM'] = cities['STATE'].str.upper().str.strip()
print(cities.head(3))


In [ ]:
# Download Census cartographic boundary shapefiles (stable URLs)
# Places (2023, 1:500k) and ZCTA5 (2020, 1:500k)
places_url = 'https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_place_500k.zip'
zcta_url = 'https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_zcta520_500k.zip'

places_zip = os.path.join(DATA_DIR, 'cb_2023_us_place_500k.zip')
zcta_zip = os.path.join(DATA_DIR, 'cb_2020_us_zcta520_500k.zip')

# Download if missing
import urllib.request
for url, dest in [(places_url, places_zip), (zcta_url, zcta_zip)]:
    if not os.path.exists(dest):
        print('Downloading', url)
        urllib.request.urlretrieve(url, dest)
        print('Saved to', dest)
    else:
        print('Exists', dest)

# Unzip
import zipfile
for dest in [places_zip, zcta_zip]:
    with zipfile.ZipFile(dest, 'r') as zf:
        zf.extractall(DATA_DIR)
print('Unzipped')


In [ ]:
# Load geodata
place_fp = [f for f in os.listdir(DATA_DIR) if f.startswith('cb_2023_us_place_500k') and f.endswith('.shp')][0]
zcta_fp = [f for f in os.listdir(DATA_DIR) if f.startswith('cb_2020_us_zcta520_500k') and f.endswith('.shp')][0]

places = gpd.read_file(os.path.join(DATA_DIR, place_fp))
zctas = gpd.read_file(os.path.join(DATA_DIR, zcta_fp))

# Normalize columns
places['NAME_NORM'] = places['NAME'].str.upper().str.replace('\.', '', regex=False)
# In cartographic shapefiles, state FIPS is in STATEFP
places['STATEFP'] = places['STATEFP']

# ZCTA code column could be ZCTA5CE10 or ZCTA5CE20; detect
zcta_col = 'ZCTA5CE10' if 'ZCTA5CE10' in zctas.columns else ('ZCTA5CE20' if 'ZCTA5CE20' in zctas.columns else None)
if zcta_col is None:
    raise RuntimeError(f'ZCTA code column not found. Columns: {list(zctas.columns)}')
zctas[zcta_col] = zctas[zcta_col].astype(str)

print(len(places), 'places;', len(zctas), 'zctas')


In [ ]:
# Map state abbreviations to FIPS
state_abbr_to_fips = {
    'AL':'01','AK':'02','AZ':'04','AR':'05','CA':'06','CO':'08','CT':'09','DE':'10','DC':'11','FL':'12','GA':'13','HI':'15','ID':'16','IL':'17','IN':'18','IA':'19','KS':'20','KY':'21','LA':'22','ME':'23','MD':'24','MA':'25','MI':'26','MN':'27','MS':'28','MO':'29','MT':'30','NE':'31','NV':'32','NH':'33','NJ':'34','NM':'35','NY':'36','NC':'37','ND':'38','OH':'39','OK':'40','OR':'41','PA':'42','RI':'44','SC':'45','SD':'46','TN':'47','TX':'48','UT':'49','VT':'50','VA':'51','WA':'53','WV':'54','WI':'55','WY':'56'}

# Build lookup on places by (name,state)
places['KEY'] = places['NAME_NORM'] + '|' + places['STATEFP']

print('Prepared place keys')


In [ ]:
# Compute ZCTA centroids in same CRS as places
if zctas.crs != places.crs:
    zctas = zctas.to_crs(places.crs)

zcta_centroids = zctas.copy()
zcta_centroids['geometry'] = zcta_centroids.geometry.centroid

# Track the chosen ZCTA code column across the notebook
zcta_code_col = zcta_col

print('Prepared centroids')


In [ ]:
# For each city, find matching place(s) and collect ZCTAs whose centroids fall within
core_rows = []

for _, r in tqdm(cities.iterrows(), total=len(cities)):
    city_name = r['CITY_NORM']
    state_abbr = r['STATE_NORM']
    statefp = state_abbr_to_fips.get(state_abbr)
    if statefp is None:
        core_rows.append({**r.to_dict(), 'ZIPCODES': ''})
        continue
    # Some city names may include qualifiers like "St. Louis" vs "Saint Louis"
    name_variants = {city_name,
                     city_name.replace('ST ', 'SAINT '),
                     city_name.replace('SAINT ', 'ST ')}

    candidate_places = places[places['STATEFP'] == statefp]
    candidate_places = candidate_places[candidate_places['NAME_NORM'].isin(name_variants)]

    if candidate_places.empty:
        # fallback: contains
        candidate_places = places[(places['STATEFP'] == statefp) & (places['NAME_NORM'].str.contains(city_name, regex=False))]

    if candidate_places.empty:
        core_rows.append({**r.to_dict(), 'ZIPCODES': ''})
        continue

    # Union geometry if multiple place parts
    union_geom = candidate_places.unary_union

    # Select ZCTAs whose centroid is within place polygon
    within_mask = zcta_centroids.within(union_geom)
    zips = zcta_centroids.loc[within_mask, zcta_code_col].tolist()
    zips_sorted = sorted(set(zips))

    rec = r.to_dict()
    rec['ZIPCODES'] = ','.join(zips_sorted)
    core_rows.append(rec)

core_df = pd.DataFrame(core_rows)
print('Done cities:', len(core_df))


In [ ]:
# Save output in requested format
output = core_df[['CITY_STATE','CITY','STATE','POPULATION','CITY_RANK']].copy()
output['ZIPCODES'] = core_df['ZIPCODES']
output.to_csv('top_100_cities_core_zipcodes.csv', index=False)
print('Wrote top_100_cities_core_zipcodes.csv')

# Quick spot checks
for city in ['Tampa', 'Orlando', 'Miami']:
    row = output[output['CITY'] == city]
    if not row.empty:
        print(city, '->', row.iloc[0]['ZIPCODES'][:120] + ('...' if len(row.iloc[0]['ZIPCODES'])>120 else ''))


In [ ]:
# Create long-form CSV with one zipcode per row
import pandas as pd

wide_df = pd.read_csv('top_100_cities_core_zipcodes.csv')
wide_df['ZIPCODES'] = wide_df['ZIPCODES'].fillna('')

# Split comma-separated zipcodes and explode to rows
long_df = wide_df.assign(ZIPCODE=wide_df['ZIPCODES'].str.split(',')).explode('ZIPCODE')
long_df['ZIPCODE'] = long_df['ZIPCODE'].astype(str).str.strip()
long_df = long_df[long_df['ZIPCODE'] != '']

# Keep requested columns
long_df = long_df[['CITY_STATE','CITY','STATE','POPULATION','CITY_RANK','ZIPCODE']]

# Save
long_out = 'top_100_cities_core_zipcodes_long.csv'
long_df.to_csv(long_out, index=False)
print('Wrote', long_out, 'rows:', len(long_df))

# Quick peek
print(long_df.head(10).to_string(index=False))
